In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import torch
import torchvision
import torch.nn as nn # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torchvision.datasets as datasets # Has standard datasets we can import in a nice way
import torchvision.transforms as transforms # Transformations we can perform on our dataset
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader, Dataset # Gives easier dataset managment and creates mini batches
from torchvision.datasets import ImageFolder
import torch.optim as optim # For all Optimization algorithms, SGD, Adam, etc.
from PIL import Image
import random
import shutil
from math import ceil
from torchvision.transforms.functional import to_tensor
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporhttps://discord.gg/9RMZspGjhttps://discord.gg/9RMZspGjhttps://discord.gg/9RMZspGjary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/microsoft-catsvsdogs-dataset/readme[1].txt
/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/7981.jpg
/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/7981.jpg


In [2]:
os.listdir('/kaggle/checkpoints')
print("remove checkpoints")
for f in os.listdir('/kaggle/checkpoints'):
    f_name = os.path.join('/kaggle/checkpoints',f)
    os.remove(f_name)
print("files in checkpoints/")
os.listdir('/kaggle/checkpoints')


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/checkpoints'

In [ ]:
os.listdir('/kaggle/checkpoints')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu or cpu
img_size = 224
orig_data_path = "/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/"
train_path = "/kaggle/dataset/train"
val_path = "/kaggle/dataset/val"
test_path = "/kaggle/dataset/test"
saved_models = "/kaggle/checkpoints"
batch_size = 64
classes = ['Dog','Cat']
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

os.mkdir(saved_models)

In [ ]:
if not os.path.isdir(saved_models):
    os.makedirs(saved_models)

In [ ]:
len(os.listdir(os.path.join(orig_data_path,"Dog")))

# Split train-val-test

In [ ]:
random.seed(42)

if os.path.isdir('/kaggle/dataset'):
    shutil.rmtree('/kaggle/dataset')
else:
    print("ok")

for target_path in [train_path, val_path, test_path]:
    if not os.path.exists(target_path):
        os.makedirs(target_path)
        for _class in classes:
            print("class: ",_class)
            os.makedirs(os.path.join(target_path, _class))

            
def is_valid_image_file(filename):
    valid_extensions = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
    return any(filename.endswith(ext) for ext in valid_extensions)
            
# split the dataset into train, test
for _class in classes:
    class_path = os.path.join(orig_data_path, _class)
    images = [img for img in os.listdir(class_path) if is_valid_image_file(img)]
    #images = os.listdir(class_path)
    random.shuffle(images)
    
    train_images = images[:ceil(len(images) * train_ratio)]
    val_images = images[ceil(len(images) * train_ratio):ceil(len(images) * (train_ratio + val_ratio))]
    test_images = images[ceil(len(images) * (train_ratio + val_ratio)):]
    
    print("Copying files...")
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_path, _class, img))
    for img in val_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_path, _class, img)) 
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_path, _class, img))                
    print("Finish")
    
    print(f"Number of train {_class} = {len(os.listdir(os.path.join(train_path, _class)))}")
    print(f"Number of val {_class} = {len(os.listdir(os.path.join(val_path, _class)))}")
    print(f"Number of test {_class} = {len(os.listdir(os.path.join(test_path, _class)))}")
print("Done")

# Data Loaders

In [ ]:
# initial checks for images
# check image not corrupted
class ImageFolderWithFilter(torchvision.datasets.ImageFolder):
    def __init__(self, root, transform=None, target_transform=None):
        super(ImageFolderWithFilter, self).__init__(root, transform=transform, target_transform=target_transform)
        self.samples = self._filter_invalid_images()

    def _filter_invalid_images(self):
        valid_samples = []
        for sample in self.samples:
            try:
                Image.open(sample[0]).convert("RGB")
                valid_samples.append(sample)
            except (IOError, SyntaxError) as e:
                print(f"Invalid image: {sample[0]} - {e}")

        return valid_samples



# check image not B&W or RGBA
class ToTensorEnsure3D:
    def __call__(self, pic):
        # Convert the image to a 3-channel image if it is grayscale or has an alpha channel
        if pic.mode == 'L' or pic.mode == 'RGBA':
            pic = pic.convert('RGB')
        return to_tensor(pic)

simple_transforms = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    ToTensorEnsure3D(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


# -----------------------------------------------------------------

# Load the datasets with ImageFolderWithFilter and apply the simple_transforms
train_dataset = ImageFolderWithFilter(train_path, transform=simple_transforms)
val_dataset = ImageFolderWithFilter(val_path, transform=simple_transforms)
test_dataset = ImageFolderWithFilter(test_path, transform=simple_transforms)

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
print("Done")

# Simple NN fully connected

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(img_size * img_size * 3, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the input
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

# Create the network
net = SimpleNet()
net.to(device)

# inspect the network

In [ ]:
!pip install torchsummary
from torchsummary import summary

# Print the model summary
summary(net, input_size=(3, img_size, img_size))

In [ ]:
img_size * img_size * 128 * 3 + 128

# Training loop

In [ ]:
import time

def train(model, nb_epochs, name_model):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    
    start = time.time()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

    for epoch in range(nb_epochs):

        epoch_loss = 0
        epoch_accuracy = 0
        
        for X, y in train_loader:
            X = X.to(device)
            y = y.to(device)
            preds = model(X)
            loss = criterion(preds, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            accuracy = ((preds.argmax(dim=1) == y).float().mean())
            epoch_accuracy += accuracy.item()
            epoch_loss += loss.item()
            
            print('.', end='', flush=True)

        epoch_accuracy = epoch_accuracy/len(train_loader)
        train_accuracies.append(epoch_accuracy)
        epoch_loss = epoch_loss / len(train_loader)
        train_losses.append(epoch_loss)

        print("\n --- Epoch: {}, train loss: {:.4f}, train acc: {:.4f}, time: {}".format(epoch, epoch_loss, epoch_accuracy, time.time() - start))
        # save model
        save_name = os.path.join(saved_models, name_model +'_checkpoint_epoch_'+str(epoch)+'.pt')
        print(f"saved checkpoint {save_name}")
        torch.save({ 
                    'model_state_dict': model.state_dict(), 
                    'optimizer_state_dict': optimizer.state_dict(), 
                    }, save_name)
        
        '''
        torch.save({ 
                    'model_state_dict': model.state_dict(), 
                    'optimizer_state_dict': optimizer.state_dict(), 
                    }, 'checpoint_epoch_'+str(epoch)+'.pt')
        '''
        # test set accuracy
        with torch.no_grad():

            val_epoch_loss = 0
            val_epoch_accuracy = 0

            for val_X, val_y in val_loader:
                val_X = val_X.to(device)
                val_y = val_y.to(device)
                val_preds = model(val_X)
                val_loss = criterion(val_preds, val_y)

                val_epoch_loss += val_loss.item()            
                val_accuracy = ((val_preds.argmax(dim=1) == val_y).float().mean())
                val_epoch_accuracy += val_accuracy.item()

            val_epoch_accuracy = val_epoch_accuracy/len(val_loader)
            val_epoch_loss = val_epoch_loss / len(val_loader)
            val_losses.append(val_epoch_loss)
            val_accuracies.append(val_epoch_accuracy)
            print("Epoch: {}, val loss: {:.4f}, val acc: {:.4f}, time: {}\n".format(epoch, val_epoch_loss, val_epoch_accuracy, time.time() - start))
            
        
    # Plot the training and validation losses
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss per Epoch')
    plt.legend()
    
    plt.plot(train_accuracies, label='Training Acc')
    plt.plot(val_accuracies, label='Validation Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Acc per Epoch')
    plt.legend()
    
    
    plt.show()

# Testing loop

In [ ]:
def test(model):
    criterion = nn.CrossEntropyLoss()
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            _, predictions = torch.max(output, 1)
            correct += (predictions == y).sum().item()
            test_loss = criterion(output, y)
            
    test_loss /= len(test_loader.dataset)
    print("Average Loss: ", test_loss, "  Accuracy: ", correct, " / ",
    len(test_loader.dataset), "  ", int(correct / len(test_loader.dataset) * 100), "%")

In [ ]:
train(net,10, 'simple_fc')

In [ ]:
test(net)

# Chenge to a CNN model

In [ ]:
class CatDogCNN(nn.Module):
    def __init__(self, img_size):
        super(CatDogCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        reduced_size = img_size // 2**3  # Calculate the reduced size after three pooling layers
        self.fc1 = nn.Linear(128 * reduced_size * reduced_size, 512)
        self.fc2 = nn.Linear(512, 2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

net = CatDogCNN(img_size)
net.to(device)

In [ ]:
train(net,10, 'our_CNN')

In [ ]:
test(net)

# A pre-trained model

In [ ]:
from tqdm import tqdm
from torchvision import models
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model.to(device)

train(model, 10, "pretrained")
test(model)

In [ ]:
def RandomImagePrediction(filepath, model):
    img_array = Image.open(filepath).convert("RGB")
    real_class = filepath.split("/")[3]
    
    
    data_transforms=transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(), 
        transforms.Normalize([0.5]*3, [0.5]*3)
    ])
    img = data_transforms(img_array).unsqueeze(dim=0) # Returns a new tensor with a dimension of size one inserted at the specified position.
    load = DataLoader(img)
    
    for x in load:
        x=x.to(device)
        pred = model(x)
        _, preds = torch.max(pred, 1)
        #print(f"class : {preds}")
        if preds[0] == 1: print(f"real class {real_class} and predicted ----> Dog")
        else: print(f"real class {real_class} and predicted ----> Cat")

In [ ]:
RandomImagePrediction("/tmp/test/Cat/11.jpg", net)
RandomImagePrediction("/tmp/test/Cat/48.jpg", net)
RandomImagePrediction("/tmp/test/Cat/75.jpg", net)
RandomImagePrediction("/tmp/test/Cat/88.jpg", net)
RandomImagePrediction("/tmp/test/Cat/104.jpg", net)
RandomImagePrediction("/tmp/test/Dog/2.jpg", net)
RandomImagePrediction("/tmp/test/Dog/6.jpg", net)
RandomImagePrediction("/tmp/test/Dog/7.jpg", net)
RandomImagePrediction("/tmp/test/Dog/11.jpg", net)
RandomImagePrediction("/tmp/test/Dog/14.jpg", net)

# Try with a pre-trained model

In [ ]:
from torchvision import models
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

model.to(device)